# HW06: Word Embeddings

Remember that these homework work as a completion grade. **In this homework, we present two tasks and you can choose which one you want to solve. You only have to solve <span style="color:red">one task</span> in this homework.**
Task 1 is more guided and we evaluate document embeddings on a standard benchmark. Task 2 is very open-end and might be a starting point for your course project.

**Task 1**
In this task, we evaluate different document embeddings on the English version of the [STS Benchmark](https://arxiv.org/pdf/1708.00055.pdf). The task is to determine how semantically similar two texts are and is a popular dataset to evaluate document embeddings, i.e. we want embeddings of two semantically similar documents to be similar as well. We provide a wordcounts baseline for this task and ask you to compute and evaluate embeddings for a selected sample of document embedding techniques.

To evaluate, we follow [(Reimers and Gurevych, 2019)](https://arxiv.org/pdf/1908.10084.pdf) and compute the Spearman’s rank correlation between the cosine-similarity of the sentence embeddings and the gold labels. **It is ok to skip one of the document embedding methods**

In [ ]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

In [ ]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

In [ ]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

In [ ]:
from nltk import word_tokenize
from random import shuffle

docs_a = []
docs_b = []

def tokenize(data):
    docs = []
    for item in data:
        docs += [word_tokenize(item)]
    return docs

docs_a = tokenize(text_a)
docs_b = tokenize(text_b)

In [ ]:
##TODO train Doc2Vec on the texts in the dataset
##TODO derive the word vectors for each text in the dataset
##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
doc_iterator_a = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs_a)]
doc_iterator_b = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs_b)]

d2v_a = Doc2Vec(doc_iterator_a, min_count = 10,
                window = 5, vector_size = 50,
                sample = 1e-4, negative = 5,
                worker = 4, max_vocab_size = 1000)


d2v_b = Doc2Vec(doc_iterator_b, min_count = 10,
                window = 5, vector_size = 50,
                sample = 1e-4, negative = 5,
                worker = 4, max_vocab_size = 1000)


In [ ]:
a = d2v_a.docvecs.vectors_docs
b = d2v_b.docvecs.vectors_docs

In [ ]:
a.shape, b.shape

In [ ]:
predictions = [cosine_similarity(t1,t2) for t1,t2 in zip(a, b)]
evaluate(predictions, labels)

In [ ]:
##TODO do the same with embeddings provided by spaCy
import spacy

nlp = spacy.load('en_core_web_sm')
documents_a = [nlp(item) for item in text_a]
documents_b = [nlp(item) for item in text_b]

In [ ]:
predictions = [cosine_similarity(t1.vector,t2.vector) for t1,t2 in zip(documents_a, documents_b)]
evaluate(predictions, labels)

In [ ]:
##TODO do the same with universal sentence embeddings
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
tf.disable_eager_execution()

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

def embed(input):
    return model(input)

In [ ]:
embeddings_a = []
embeddings_b = []

text_ue_a = []
text_ue_b = []

for i, item in enumerate(text_a):
    embeddings_a.append(embed([item]))
    
for i, item in enumerate(text_b):
    embeddings_b.append(embed([item]))

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    for item in embeddings_a:
        text_ue_a.append(session.run(item))
    for item in embeddings_b:
        text_ue_b.append(session.run(item))

In [ ]:
predictions = [cosine_similarity(t1[0],t2[0]) for t1,t2 in zip(text_ue_a, text_ue_b)]
evaluate(predictions, labels)

In [ ]:
##TODO do the same with SBERT embeddings
from sentence_transformers import SentenceTransformer
model = "bert-base-nli-mean-tokens"
embedder = SentenceTransformer(model)
text_encoded_a = []
text_encoded_b = []
for i, item in enumerate(text_a):
    text_encoded_a.append(embedder.encode(item))
    
for i, item in enumerate(text_b):
    text_encoded_b.append(embedder.encode(item))

In [ ]:
predictions = [cosine_similarity(t1,t2) for t1,t2 in zip(text_encoded_a, text_encoded_b)]
evaluate(predictions, labels)

**Task 2**
Use your favorite document embeddings method to compute embeddings for a dataset you are interested in. Think of a method and provide some data visualization statistics (one method would be the path we have chosen in the notebook, i.e. cluster the embeddings with k-means and visualize low-dimensional representations of the document embeddings obtained by PCA). 

This task is very open and there is no right or wrong; If you want to use document embeddings in your course project, this is a chance to play around with them.

